In [1]:
!pip install numpy pandas datasets sacrebleu bert_score torchmetrics rouge_score transformers tqdm matplotlib seaborn torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.1 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=024a9ea35a312de5f22090b8f418db5da45b5e74fd2f2f8718f2dac8ec66e2e8
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
import numpy as np
import pandas as pd
from datasets import load_dataset
from sacrebleu.metrics import BLEU, CHRF, TER
from bert_score import BERTScorer
from torchmetrics.text import TranslationEditRate, WordErrorRate, CharErrorRate
from rouge_score import rouge_scorer
from transformers import AutoTokenizer, AutoModelForCausalLM
from tqdm import tqdm
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import torch

warnings.filterwarnings('ignore')

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define models
model_names = ["Qwen/Qwen2.5-0.5B-Instruct"]

def load_translation_data(language_pair, num_samples=100):
    """Load dataset for specified language pair."""
    try:
        dataset = load_dataset("wmt19", language_pair, split="validation")
    except ValueError:
        dataset = load_dataset("wmt19", language_pair, split="train")
    return dataset.select(range(min(num_samples, len(dataset))))

def translate_text(text, source_lang, target_lang, model_name, tokenizer, model):
    """Translate text using the specified model."""
    prompt = f"Translate the following text from {source_lang} to {target_lang}: {text}"
    inputs = tokenizer(prompt, return_tensors="pt", max_length=256, truncation=True).to(device)
    
    # Generate translation with no gradients
    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=256)
    return tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

def calculate_metrics(references, hypotheses, target_lang):
    """Calculate various MT evaluation metrics."""
    bleu_tokenizer = "zh" if target_lang == "zh" else "13a"
    bleu = BLEU(tokenizer=bleu_tokenizer)
    chrf = CHRF()
    ter_metric = TER()
    bert_scorer = BERTScorer(lang="en", rescale_with_baseline=True)
    wer = WordErrorRate()
    cer = CharErrorRate()
    rouge_metrics = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    
    bleu_score = bleu.corpus_score(hypotheses, [references]).score
    chrf_score = chrf.corpus_score(hypotheses, [references]).score
    ter_score = ter_metric.corpus_score(hypotheses, [references]).score
    P, R, F1 = bert_scorer.score(hypotheses, references)
    bert_score = F1.mean().item()
    wer_score = wer(hypotheses, references).item()
    cer_score = cer(hypotheses, references).item()
    rouge_scores = {'rouge1': 0, 'rouge2': 0, 'rougeL': 0}
    
    for hyp, ref in zip(hypotheses, references):
        scores = rouge_metrics.score(ref, hyp)
        rouge_scores['rouge1'] += scores['rouge1'].fmeasure
        rouge_scores['rouge2'] += scores['rouge2'].fmeasure
        rouge_scores['rougeL'] += scores['rougeL'].fmeasure
    
    for key in rouge_scores:
        rouge_scores[key] /= len(hypotheses)

    return {
        "BLEU": bleu_score,
        "chrF": chrf_score,
        "TER": ter_score,
        "BERTScore": bert_score,
        "WER": wer_score,
        "CER": cer_score,
        "ROUGE-1": rouge_scores['rouge1'],
        "ROUGE-2": rouge_scores['rouge2'],
        "ROUGE-L": rouge_scores['rougeL']
    }

def plot_confusion_matrix(references, translations, model_name, pair_code):
    """Generate and save a confusion matrix."""
    matrix = np.zeros((len(references), len(translations)))
    for i, ref in enumerate(references):
        for j, hyp in enumerate(translations):
            matrix[i, j] = len(set(ref.split()) & set(hyp.split())) / len(set(ref.split()) | set(hyp.split()))
    
    plt.figure(figsize=(12, 8))
    sns.heatmap(matrix, annot=False, cmap="coolwarm", cbar=True)
    plt.title(f"Confusion Matrix for {model_name} ({pair_code})")
    plt.xlabel("Hypotheses")
    plt.ylabel("References")
    plt.tight_layout()
    plt.savefig(f"confusion_matrix_{model_name}_{pair_code}.png")
    plt.close()

def evaluate_model(dataset, source_lang, target_lang, model_name):
    """Evaluate a specific model on the dataset."""
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
    
    translations = []
    references = []
    source_texts = []
    
    for example in tqdm(dataset, desc=f"Translating with {model_name}"):
        source_text = example['translation'][source_lang]
        reference = example['translation'][target_lang]
        
        try:
            translation = translate_text(source_text, source_lang, target_lang, model_name, tokenizer, model)
            source_texts.append(source_text)
            translations.append(translation)
            references.append(reference)
        except Exception as e:
            print(f"Error during translation with {model_name}: {str(e)}")
            continue

    # Clear GPU memory after processing
    model.to("cpu")
    torch.cuda.empty_cache()

    if translations:
        metrics = calculate_metrics(references, translations)
        return source_texts, references, translations, metrics
    return [], [], [], {}

def main():
    language_pairs = [
        ("cs-en", "Czech-English"),
        ("de-en", "German-English"),
        ("fi-en", "Finnish-English"),
        ("fr-de", "French-German"),
        ("gu-en", "Gujarati-English"),
        ("kk-en", "Kazakh-English"),
        ("lt-en", "Lithuanian-English"),
        ("ru-en", "Russian-English"),
        ("zh-en", "Chinese-English")
    ]
    
    for pair_code, pair_name in language_pairs:
        print(f"\nEvaluating {pair_name} translations...")
        dataset = load_translation_data(pair_code, num_samples=100)
        source_lang, target_lang = pair_code.split("-")
        
        for model_name in model_names:
            # Original Direction
            print(f"Evaluating {model_name} in original direction ({source_lang} -> {target_lang})...")
            source_texts, references, translations, metrics = evaluate_model(dataset, source_lang, target_lang, model_name)
            
            # Save metrics
            results_df = pd.DataFrame([metrics], index=[model_name.split("/")[-1]])
            results_df.to_csv(f"mt_evaluation_results_{model_name.split('/')[-1]}_{pair_code}_original.csv")
            
            # Save translations for review
            translations_df = pd.DataFrame({
                "Source": source_texts,
                "Reference": references,
                "Translation": translations
            })
            translations_df.to_csv(f"translations_{model_name.split('/')[-1]}_{pair_code}_original.csv", index=False)
            
            # Plot and save confusion matrix
            # plot_confusion_matrix(references, translations, model_name.split("/")[-1], f"{pair_code}_original")
            
            # Print metrics
            print(f"\nResults for {model_name} ({pair_name}) in original direction:")
            print(results_df)

            # Reverse Direction
            print(f"Evaluating {model_name} in reverse direction ({target_lang} -> {source_lang})...")
            source_texts, references, translations, metrics = evaluate_model(dataset, target_lang, source_lang, model_name)
            
            # Save metrics
            results_df = pd.DataFrame([metrics], index=[model_name.split("/")[-1]])
            results_df.to_csv(f"mt_evaluation_results_{model_name.split('/')[-1]}_{pair_code}_reverse.csv")
            
            # Save translations for review
            translations_df = pd.DataFrame({
                "Source": source_texts,
                "Reference": references,
                "Translation": translations
            })
            translations_df.to_csv(f"translations_{model_name.split('/')[-1]}_{pair_code}_reverse.csv", index=False)
            
            # Plot and save confusion matrix
            # plot_confusion_matrix(references, translations, model_name.split("/")[-1], f"{pair_code}_reverse")
            
            # Print metrics
            print(f"\nResults for {model_name} ({pair_name}) in reverse direction:")
            print(results_df)



if __name__ == "__main__":
    main()


Using device: cuda

Evaluating Czech-English translations...


README.md:   0%|          | 0.00/11.3k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/195M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/216M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/254M [00:00<?, ?B/s]

...

Evaluating Qwen/Qwen2.5-0.5B-Instruct in original direction (zh -> en)...


Translating with Qwen/Qwen2.5-0.5B-Instruct: 100%|██████████| 100/100 [09:01<00:00,  5.41s/it]
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Results for Qwen/Qwen2.5-0.5B-Instruct (Chinese-English) in original direction:
                           BLEU       chrF         TER  BERTScore       WER  \
Qwen2.5-0.5B-Instruct  3.055662  28.867378  444.358974  -0.245966  4.483333   

                            CER   ROUGE-1   ROUGE-2   ROUGE-L  
Qwen2.5-0.5B-Instruct  4.586035  0.219233  0.076213  0.155504  
Evaluating Qwen/Qwen2.5-0.5B-Instruct in reverse direction (en -> zh)...


Translating with Qwen/Qwen2.5-0.5B-Instruct: 100%|██████████| 100/100 [08:33<00:00,  5.14s/it]
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Results for Qwen/Qwen2.5-0.5B-Instruct (Chinese-English) in reverse direction:
                           BLEU      chrF          TER  BERTScore        WER  \
Qwen2.5-0.5B-Instruct  0.042384  5.335408  9906.293706  -0.110036  99.062935   

                             CER   ROUGE-1   ROUGE-2  ROUGE-L  
Qwen2.5-0.5B-Instruct  17.551081  0.013815  0.003908  0.01372  


In [3]:
import numpy as np
import pandas as pd
from datasets import load_dataset
from sacrebleu.metrics import BLEU, CHRF, TER
from bert_score import BERTScorer
from torchmetrics.text import TranslationEditRate, WordErrorRate, CharErrorRate
from rouge_score import rouge_scorer
from transformers import AutoTokenizer, AutoModelForCausalLM
from tqdm import tqdm
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import torch

warnings.filterwarnings('ignore')

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define models
model_names = ["Qwen/Qwen2.5-1.5B-Instruct"]

def load_translation_data(language_pair, num_samples=100):
    """Load dataset for specified language pair."""
    try:
        dataset = load_dataset("wmt19", language_pair, split="validation")
    except ValueError:
        dataset = load_dataset("wmt19", language_pair, split="train")
    return dataset.select(range(min(num_samples, len(dataset))))

def translate_text(text, source_lang, target_lang, model_name, tokenizer, model):
    """Translate text using the specified model."""
    prompt = f"Translate the following text from {source_lang} to {target_lang}: {text}"
    inputs = tokenizer(prompt, return_tensors="pt", max_length=256, truncation=True).to(device)
    
    # Generate translation with no gradients
    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=256)
    return tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

def calculate_metrics(references, hypotheses):
    """Calculate various MT evaluation metrics."""
    bleu = BLEU()
    chrf = CHRF()
    ter_metric = TER()
    bert_scorer = BERTScorer(lang="en", rescale_with_baseline=True)
    wer = WordErrorRate()
    cer = CharErrorRate()
    rouge_metrics = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    
    bleu_score = bleu.corpus_score(hypotheses, [references]).score
    chrf_score = chrf.corpus_score(hypotheses, [references]).score
    ter_score = ter_metric.corpus_score(hypotheses, [references]).score
    P, R, F1 = bert_scorer.score(hypotheses, references)
    bert_score = F1.mean().item()
    wer_score = wer(hypotheses, references).item()
    cer_score = cer(hypotheses, references).item()
    rouge_scores = {'rouge1': 0, 'rouge2': 0, 'rougeL': 0}
    
    for hyp, ref in zip(hypotheses, references):
        scores = rouge_metrics.score(ref, hyp)
        rouge_scores['rouge1'] += scores['rouge1'].fmeasure
        rouge_scores['rouge2'] += scores['rouge2'].fmeasure
        rouge_scores['rougeL'] += scores['rougeL'].fmeasure
    
    for key in rouge_scores:
        rouge_scores[key] /= len(hypotheses)

    return {
        "BLEU": bleu_score,
        "chrF": chrf_score,
        "TER": ter_score,
        "BERTScore": bert_score,
        "WER": wer_score,
        "CER": cer_score,
        "ROUGE-1": rouge_scores['rouge1'],
        "ROUGE-2": rouge_scores['rouge2'],
        "ROUGE-L": rouge_scores['rougeL']
    }

def plot_confusion_matrix(references, translations, model_name, pair_code):
    """Generate and save a confusion matrix."""
    matrix = np.zeros((len(references), len(translations)))
    for i, ref in enumerate(references):
        for j, hyp in enumerate(translations):
            matrix[i, j] = len(set(ref.split()) & set(hyp.split())) / len(set(ref.split()) | set(hyp.split()))
    
    plt.figure(figsize=(12, 8))
    sns.heatmap(matrix, annot=False, cmap="coolwarm", cbar=True)
    plt.title(f"Confusion Matrix for {model_name} ({pair_code})")
    plt.xlabel("Hypotheses")
    plt.ylabel("References")
    plt.tight_layout()
    plt.savefig(f"confusion_matrix_{model_name}_{pair_code}.png")
    plt.close()

def evaluate_model(dataset, source_lang, target_lang, model_name):
    """Evaluate a specific model on the dataset."""
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
    
    translations = []
    references = []
    source_texts = []
    
    for example in tqdm(dataset, desc=f"Translating with {model_name}"):
        source_text = example['translation'][source_lang]
        reference = example['translation'][target_lang]
        
        try:
            translation = translate_text(source_text, source_lang, target_lang, model_name, tokenizer, model)
            source_texts.append(source_text)
            translations.append(translation)
            references.append(reference)
        except Exception as e:
            print(f"Error during translation with {model_name}: {str(e)}")
            continue

    # Clear GPU memory after processing
    model.to("cpu")
    torch.cuda.empty_cache()

    if translations:
        metrics = calculate_metrics(references, translations)
        return source_texts, references, translations, metrics
    return [], [], [], {}

def main():
    language_pairs = [
        ("cs-en", "Czech-English"),
        ("de-en", "German-English"),
        ("fi-en", "Finnish-English"),
        ("fr-de", "French-German"),
        ("gu-en", "Gujarati-English"),
        ("kk-en", "Kazakh-English"),
        ("lt-en", "Lithuanian-English"),
        ("ru-en", "Russian-English"),
        ("zh-en", "Chinese-English")
    ]
    
    for pair_code, pair_name in language_pairs:
        print(f"\nEvaluating {pair_name} translations...")
        dataset = load_translation_data(pair_code, num_samples=100)
        source_lang, target_lang = pair_code.split("-")
        
        for model_name in model_names:
            # Original Direction
            print(f"Evaluating {model_name} in original direction ({source_lang} -> {target_lang})...")
            source_texts, references, translations, metrics = evaluate_model(dataset, source_lang, target_lang, model_name)
            
            # Save metrics
            results_df = pd.DataFrame([metrics], index=[model_name.split("/")[-1]])
            results_df.to_csv(f"mt_evaluation_results_{model_name.split('/')[-1]}_{pair_code}_original.csv")
            
            # Save translations for review
            translations_df = pd.DataFrame({
                "Source": source_texts,
                "Reference": references,
                "Translation": translations
            })
            translations_df.to_csv(f"translations_{model_name.split('/')[-1]}_{pair_code}_original.csv", index=False)
            
            # Plot and save confusion matrix
            # plot_confusion_matrix(references, translations, model_name.split("/")[-1], f"{pair_code}_original")
            
            # Print metrics
            print(f"\nResults for {model_name} ({pair_name}) in original direction:")
            print(results_df)

            # Reverse Direction
            print(f"Evaluating {model_name} in reverse direction ({target_lang} -> {source_lang})...")
            source_texts, references, translations, metrics = evaluate_model(dataset, target_lang, source_lang, model_name)
            
            # Save metrics
            results_df = pd.DataFrame([metrics], index=[model_name.split("/")[-1]])
            results_df.to_csv(f"mt_evaluation_results_{model_name.split('/')[-1]}_{pair_code}_reverse.csv")
            
            # Save translations for review
            translations_df = pd.DataFrame({
                "Source": source_texts,
                "Reference": references,
                "Translation": translations
            })
            translations_df.to_csv(f"translations_{model_name.split('/')[-1]}_{pair_code}_reverse.csv", index=False)
            
            # Plot and save confusion matrix
            # plot_confusion_matrix(references, translations, model_name.split("/")[-1], f"{pair_code}_reverse")
            
            # Print metrics
            print(f"\nResults for {model_name} ({pair_name}) in reverse direction:")
            print(results_df)



if __name__ == "__main__":
    main()

Using device: cuda

Evaluating Czech-English translations...
Evaluating Qwen/Qwen2.5-1.5B-Instruct in original direction (cs -> en)...


tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

...


Results for Qwen/Qwen2.5-1.5B-Instruct (Russian-English) in reverse direction:
                           BLEU       chrF         TER  BERTScore       WER  \
Qwen2.5-1.5B-Instruct  0.910769  13.020258  942.539338  -0.080519  9.427021   

                            CER   ROUGE-1   ROUGE-2   ROUGE-L  
Qwen2.5-1.5B-Instruct  7.782375  0.003668  0.000678  0.003668  

Evaluating Chinese-English translations...
Evaluating Qwen/Qwen2.5-1.5B-Instruct in original direction (zh -> en)...


Translating with Qwen/Qwen2.5-1.5B-Instruct: 100%|██████████| 100/100 [10:10<00:00,  6.11s/it]
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Results for Qwen/Qwen2.5-1.5B-Instruct (Chinese-English) in original direction:
                           BLEU       chrF         TER  BERTScore       WER  \
Qwen2.5-1.5B-Instruct  4.941835  32.362379  351.474359  -0.288602  3.564423   

                            CER   ROUGE-1   ROUGE-2   ROUGE-L  
Qwen2.5-1.5B-Instruct  3.941724  0.276056  0.120375  0.212749  
Evaluating Qwen/Qwen2.5-1.5B-Instruct in reverse direction (en -> zh)...


Translating with Qwen/Qwen2.5-1.5B-Instruct: 100%|██████████| 100/100 [09:56<00:00,  5.96s/it]
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Results for Qwen/Qwen2.5-1.5B-Instruct (Chinese-English) in reverse direction:
                           BLEU     chrF          TER  BERTScore         WER  \
Qwen2.5-1.5B-Instruct  0.050823  4.32966  10720.27972  -0.227371  107.202797   

                             CER   ROUGE-1   ROUGE-2   ROUGE-L  
Qwen2.5-1.5B-Instruct  18.483522  0.014194  0.005341  0.014099  


In [4]:
import numpy as np
import pandas as pd
from datasets import load_dataset
from sacrebleu.metrics import BLEU, CHRF, TER
from bert_score import BERTScorer
from torchmetrics.text import TranslationEditRate, WordErrorRate, CharErrorRate
from rouge_score import rouge_scorer
from transformers import AutoTokenizer, AutoModelForCausalLM
from tqdm import tqdm
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import torch

warnings.filterwarnings('ignore')

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define models
model_names = ["Qwen/Qwen2.5-3B-Instruct"]

def load_translation_data(language_pair, num_samples=100):
    """Load dataset for specified language pair."""
    try:
        dataset = load_dataset("wmt19", language_pair, split="validation")
    except ValueError:
        dataset = load_dataset("wmt19", language_pair, split="train")
    return dataset.select(range(min(num_samples, len(dataset))))

def translate_text(text, source_lang, target_lang, model_name, tokenizer, model):
    """Translate text using the specified model."""
    prompt = f"Translate the following text from {source_lang} to {target_lang}: {text}"
    inputs = tokenizer(prompt, return_tensors="pt", max_length=256, truncation=True).to(device)
    
    # Generate translation with no gradients
    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=256)
    return tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

def calculate_metrics(references, hypotheses):
    """Calculate various MT evaluation metrics."""
    bleu = BLEU()
    chrf = CHRF()
    ter_metric = TER()
    bert_scorer = BERTScorer(lang="en", rescale_with_baseline=True)
    wer = WordErrorRate()
    cer = CharErrorRate()
    rouge_metrics = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    
    bleu_score = bleu.corpus_score(hypotheses, [references]).score
    chrf_score = chrf.corpus_score(hypotheses, [references]).score
    ter_score = ter_metric.corpus_score(hypotheses, [references]).score
    P, R, F1 = bert_scorer.score(hypotheses, references)
    bert_score = F1.mean().item()
    wer_score = wer(hypotheses, references).item()
    cer_score = cer(hypotheses, references).item()
    rouge_scores = {'rouge1': 0, 'rouge2': 0, 'rougeL': 0}
    
    for hyp, ref in zip(hypotheses, references):
        scores = rouge_metrics.score(ref, hyp)
        rouge_scores['rouge1'] += scores['rouge1'].fmeasure
        rouge_scores['rouge2'] += scores['rouge2'].fmeasure
        rouge_scores['rougeL'] += scores['rougeL'].fmeasure
    
    for key in rouge_scores:
        rouge_scores[key] /= len(hypotheses)

    return {
        "BLEU": bleu_score,
        "chrF": chrf_score,
        "TER": ter_score,
        "BERTScore": bert_score,
        "WER": wer_score,
        "CER": cer_score,
        "ROUGE-1": rouge_scores['rouge1'],
        "ROUGE-2": rouge_scores['rouge2'],
        "ROUGE-L": rouge_scores['rougeL']
    }

def plot_confusion_matrix(references, translations, model_name, pair_code):
    """Generate and save a confusion matrix."""
    matrix = np.zeros((len(references), len(translations)))
    for i, ref in enumerate(references):
        for j, hyp in enumerate(translations):
            matrix[i, j] = len(set(ref.split()) & set(hyp.split())) / len(set(ref.split()) | set(hyp.split()))
    
    plt.figure(figsize=(12, 8))
    sns.heatmap(matrix, annot=False, cmap="coolwarm", cbar=True)
    plt.title(f"Confusion Matrix for {model_name} ({pair_code})")
    plt.xlabel("Hypotheses")
    plt.ylabel("References")
    plt.tight_layout()
    plt.savefig(f"confusion_matrix_{model_name}_{pair_code}.png")
    plt.close()

def evaluate_model(dataset, source_lang, target_lang, model_name):
    """Evaluate a specific model on the dataset."""
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
    
    translations = []
    references = []
    source_texts = []
    
    for example in tqdm(dataset, desc=f"Translating with {model_name}"):
        source_text = example['translation'][source_lang]
        reference = example['translation'][target_lang]
        
        try:
            translation = translate_text(source_text, source_lang, target_lang, model_name, tokenizer, model)
            source_texts.append(source_text)
            translations.append(translation)
            references.append(reference)
        except Exception as e:
            print(f"Error during translation with {model_name}: {str(e)}")
            continue

    # Clear GPU memory after processing
    model.to("cpu")
    torch.cuda.empty_cache()

    if translations:
        metrics = calculate_metrics(references, translations)
        return source_texts, references, translations, metrics
    return [], [], [], {}

def main():
    language_pairs = [
        ("cs-en", "Czech-English"),
        ("de-en", "German-English"),
        ("fi-en", "Finnish-English"),
        ("fr-de", "French-German"),
        ("gu-en", "Gujarati-English"),
        ("kk-en", "Kazakh-English"),
        ("lt-en", "Lithuanian-English"),
        ("ru-en", "Russian-English"),
        ("zh-en", "Chinese-English")
    ]
    
    for pair_code, pair_name in language_pairs:
        print(f"\nEvaluating {pair_name} translations...")
        dataset = load_translation_data(pair_code, num_samples=100)
        source_lang, target_lang = pair_code.split("-")
        
        for model_name in model_names:
            # Original Direction
            print(f"Evaluating {model_name} in original direction ({source_lang} -> {target_lang})...")
            source_texts, references, translations, metrics = evaluate_model(dataset, source_lang, target_lang, model_name)
            
            # Save metrics
            results_df = pd.DataFrame([metrics], index=[model_name.split("/")[-1]])
            results_df.to_csv(f"mt_evaluation_results_{model_name.split('/')[-1]}_{pair_code}_original.csv")
            
            # Save translations for review
            translations_df = pd.DataFrame({
                "Source": source_texts,
                "Reference": references,
                "Translation": translations
            })
            translations_df.to_csv(f"translations_{model_name.split('/')[-1]}_{pair_code}_original.csv", index=False)
            
            # Plot and save confusion matrix
            # plot_confusion_matrix(references, translations, model_name.split("/")[-1], f"{pair_code}_original")
            
            # Print metrics
            print(f"\nResults for {model_name} ({pair_name}) in original direction:")
            print(results_df)

            # Reverse Direction
            print(f"Evaluating {model_name} in reverse direction ({target_lang} -> {source_lang})...")
            source_texts, references, translations, metrics = evaluate_model(dataset, target_lang, source_lang, model_name)
            
            # Save metrics
            results_df = pd.DataFrame([metrics], index=[model_name.split("/")[-1]])
            results_df.to_csv(f"mt_evaluation_results_{model_name.split('/')[-1]}_{pair_code}_reverse.csv")
            
            # Save translations for review
            translations_df = pd.DataFrame({
                "Source": source_texts,
                "Reference": references,
                "Translation": translations
            })
            translations_df.to_csv(f"translations_{model_name.split('/')[-1]}_{pair_code}_reverse.csv", index=False)
            
            # Plot and save confusion matrix
            # plot_confusion_matrix(references, translations, model_name.split("/")[-1], f"{pair_code}_reverse")
            
            # Print metrics
            print(f"\nResults for {model_name} ({pair_name}) in reverse direction:")
            print(results_df)



if __name__ == "__main__":
    main()

Using device: cuda

Evaluating Czech-English translations...
Evaluating Qwen/Qwen2.5-3B-Instruct in original direction (cs -> en)...


tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

...

Translating with Qwen/Qwen2.5-3B-Instruct: 100%|██████████| 100/100 [11:25<00:00,  6.85s/it]
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Results for Qwen/Qwen2.5-3B-Instruct (Chinese-English) in original direction:
                         BLEU       chrF         TER  BERTScore       WER  \
Qwen2.5-3B-Instruct  6.591684  36.634471  333.685897  -0.226839  3.396154   

                          CER   ROUGE-1   ROUGE-2  ROUGE-L  
Qwen2.5-3B-Instruct  3.634372  0.308377  0.154974  0.24984  
Evaluating Qwen/Qwen2.5-3B-Instruct in reverse direction (en -> zh)...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Translating with Qwen/Qwen2.5-3B-Instruct: 100%|██████████| 100/100 [11:25<00:00,  6.85s/it]
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Results for Qwen/Qwen2.5-3B-Instruct (Chinese-English) in reverse direction:
                         BLEU       chrF          TER  BERTScore        WER  \
Qwen2.5-3B-Instruct  0.114036  12.633133  5425.174825   0.137431  54.251747   

                           CER   ROUGE-1   ROUGE-2   ROUGE-L  
Qwen2.5-3B-Instruct  11.491944  0.025097  0.011507  0.024933  
